<center><h1>Bayesian Workflows with CmdStanPy:<br> 
  2019 FIFA Women's World Cup</h1></center><br>
​
​
<h2> Table of Contents:<a name="TOC"></a></h2>

- [0. Overview](#overview)
- [1. The Data](#data)
- [2. The Model](#model)
- [3. Setup CmdStanPy Package and WWC Data](#IDA)
- [4. Input data as Python `dict`](#StanInput)
- [5. Compile, Fit the Model](#StanModel)
- [6. Model Checking](#checks)
- [7. Visualizations!](#Plots)


## 0. Overview<a name="overview"></a> 

The Bayesian workflow is one of model expansion and model comparison.  Therefore, we will start by creating the simplest possible model we can with the available data.

Workflow review:

- Data gathering, (preliminary data analysis)
- Build the full joint probability model - use everything you know about the world and the data
- Fit data to model (using Stan!)
- Evaluate the fit:
   + how good is the fit?
   + do the predictions make sense?
   + how sensitive are the results to the modeling assumptions?

## 1. The Data<a name="data"></a>

The World Cup is a situation where we don't have much or any historical data on the events we want to make predictions for.

We downloaded this data from [538 by Nate Silver](https://fivethirtyeight.com/methodology/how-our-club-soccer-predictions-work/)

### Raw data

* The number of teams participating: 24
* For each of the 24 teams, a pre-world cup estimate of their ability as a Soccer Power Index (spi) score
* The number of matches played up through the end of the quarterfinals round:  48
* For each match, a record containing:
    + identity of team 1
    + identity of team 2
    + goals scored by team 1
    + goals scored by team 2

The data is in the csv files:

* [womens_world_cup_2019.csv](../data/womens_world_cup_2019.csv)
* [country_prior.csv](../data/country_prior.csv)

### Stan code

We code this information in Stan as follows:
```
data {
  int I;   // number of teams
  int N;   // number of matches
  vector[I] spi_std;  // per-team ranking
  // this is a 4-column data table of per-game outcomes
  int team_1[N];
  int team_2[N];
  vector[N] score_1;
  vector[N] score_2;
}
```

### Transformed data

We transform the per-team number of goals scored into a single "score differential": 

$y = \hbox{score}_1 - \hbox{score}_2$

i.e., if $y > 0$, Team 1 won, if $y == 0$ the game was a tie, if $y < 0$, Team 1 lost.

### Stan code

We code this information in Stan as follows:
```
transformed data {
  vector[N] y = score_1 - score_2;  // "modeled" data
}
```

## 2. The Model<a name="Model"></a>

The model we use is inspired by Andrew Gelman's [blog post](https://statmodeling.stat.columbia.edu/2014/07/13/stan-analyzes-world-cup-data/) that he wrote for the FIFA Men's World Cup 2014 games as an exercise in Stan modeling.  This is a very basic model.  We don't recommend using for betting, (unless you're betting against a non-Bayesian).

The model is a variant of a common model used to infer team abilities called the [Bradley-Terry](https://github.com/stan-dev/example-models/blob/master/knitr/bradley-terry/team.stan) model where each team has an estimated ability modeled as the expected number* of goals that they will score per game.  The difference between team abilities predicts who will win the match.

(*modeled as a continuous value, which isn't correct)

### Model Parameters

To model the ability of each team we set up a regression model where all teams have a common intercept term and each team has its own slope.  We set this up in Stan as follows:

```
parameters {
  real beta;            // common intercept
  vector[I] alpha;   // vector of per-team weights
  real<lower=0> sigma_a;   // common variance
  real<lower=0> sigma_y;  // noise term in our estimate
}
transformed parameters {
  // "mixed effects" model - common intercept + random effects
  vector[I] ability = beta * spi_std + alpha * sigma_a;
}
```

### Likelihood and priors

The essential part of this model in Stan is:
```
y ~ student_t(7, ability[team_1] - ability[team_2], sigma_y);
```

This statement says that `y`, (the number of goals scored by team 1 - the number of goals scored by team 2)
is drawn from a Student_t distribution with 7 degrees of freedom.


The Stan model block is:
```
model {
  y ~ student_t(7, ability[team_1] - ability[team_2], sigma_y);

  alpha ~ normal(0, 1); // priors on all parameters
  beta ~ normal(0, 2.5);
  sigma_a ~ normal(0, 2.5);
  sigma_y ~ normal(0, 2.5);
}
```

## 3. Importing Packages + Data <a name="IDA"></a>

### Install CmdStanPy

Option 1:  install CmdStanPy using `pip`

In [ ]:
!pip install cmdstanpy

Option 2:  clone github repo:

In [ ]:
!git clone https://github.com/stan-dev/cmdstanpy.git

### Install CmdStan

In [ ]:
import cmdstanpy
cmdstanpy.install_cmdstan()

## 4. Input Data as Python `dict` <a name="StanInput"></a>

The input data for the model needs to be in a dictionary format with the following keys: <br>

1. I: The number of teams <br>
2. N: The number of matches played <br>
3. team_1: A column of the "first" team <br>
4. team_2: A column of the "second" team <br>
5. score_1: Goals scored by the first team <br>
6. score_2: Goals scored by the second team <br>
7. prior_score: The countries' prior scores (i.e., spi's before the tournament) sorted in the descending order <br>

### Get the world cup data

In [ ]:
!git clone https://github.com/nyc-pyladies/2019-cmdstanpy-bayesian-workshop

### We have two data files; The first one gives us information upto the quarterfinals:

In [ ]:
import numpy as np
import pandas as pd

matches = pd.read_csv('2019-cmdstanpy-bayesian-workshop/data/womens_world_cup_2019.csv')
matches.head()

The second one is a list of the countries that participated in the decreasing order of "ability" prior to the tournament. 
This would be our **PRIOR** here.
We use the *soccer power index*, abbreviated here as "spi", as determined at [538 by Nate Silver](https://fivethirtyeight.com/methodology/how-our-club-soccer-predictions-work/). In the stan model, these feature as "prior_score" in the input and as "ability" in the model.

In [ ]:
countries = pd.read_csv('2019-cmdstanpy-bayesian-workshop/data/country_prior.csv')
countries.head()

In [ ]:
mean = np.mean(countries['spi'])
std = np.std(countries['spi'])

#Rescaling the prior!
countries['spi_std'] = [(x - mean)/std for x in countries['spi']]
countries.head()

In [ ]:
country_mapping = countries.country.to_dict()
country_mapping = {k:v+1 for v,k in country_mapping.items()}

In [ ]:
N = len(matches)
I = len(countries)
mydict = dict({'I': I,
               'N': N,
                'team_1': matches['team_1'].values,
               'team_2': matches['team_2'].values,
               'score_1': matches['score_1'].values,
               'score_2': matches['score_2'].values,
              'spi_std': countries['spi_std'].values})

We're now going to refer to the teams by a number that represents their pre-tournament ranking. To do that we first define a dictionary where the keys are the country names and the values are the team ranking.

In [ ]:
#Replace team names by their rankings;

mydict['team_1'] = [ country_mapping.get(x) for x in mydict['team_1']]
mydict['team_2'] = [ country_mapping.get(x) for x in mydict['team_2']]

Our data is now prepped for Stan. We are ready to model it!

## 5. Compile and Fit the Stan Model<a name="StanModel"></a>

In [ ]:
import cmdstanpy
from cmdstanpy import Model, StanFit

stan_wc = Model(stan_file='2019-cmdstanpy-bayesian-workshop/models/worldcup_pyladies.stan')
stan_wc.compile()
stan_wc

We can now fit the model to the data!

In [ ]:
worldcup_fit = stan_wc.sample(data=mydict, chains=4)

## 6. Model Checking <a name="checks"></a>

In [ ]:
worldcup_fit.diagnose()

The Stanfit "summary" method provides summaries of the estimates for all parameters in the model.

In [ ]:
df = worldcup_fit.summary().round(decimals = 2)
df

## 7. Interpretation, Visualization<a name="plots"></a>


In [ ]:
dft = df.transpose()
import cmdstanpy

ability_filter = [col for col in dft if col.startswith('ability')]
yrep_filter = [col for col in dft if col.startswith('y_rep')]

#Abilities
abilities = dft[ability_filter]
abilities

In [ ]:
#Team Differentials
td = dft[yrep_filter]
td

### Let's Make Some Plots!

In [ ]:
import matplotlib.pyplot as plt
df_teamdifferentials = pd.DataFrame({'midway': td.loc['50%'].values,
                                       'names':matches['match_list']})
df_teamdifferentials.loc[:, 'left'] = td.loc['5%'].values
df_teamdifferentials.loc[:, 'right'] = td.loc['95%'].values

In [ ]:
actual_differentials = np.array([matches['score_1'][i]-matches['score_2'][i]  for i in range(len(matches))])

In [ ]:
import os
os.chdir('2019-cmdstanpy-bayesian-workshop/')
from coefplot import coefficient_plot

coefficient_plot(df_teamdifferentials['midway'], df_teamdifferentials['left'], 
                 df_teamdifferentials['right'], actual_differentials, 
                 names=df_teamdifferentials['names'],
                 title='Game Differentials Coefficient Plot', 
                fig_size = (8,12))
plt.tight_layout()

In [ ]:
df_abilities = pd.DataFrame({'midway': abilities.loc['50%'].values,
                                       'names': countries['country']})
df_abilities.loc[:, 'left'] = abilities.loc['5%'].values
df_abilities.loc[:, 'right'] = abilities.loc['95%'].values

In [ ]:
coefficient_plot(df_abilities['midway'], df_abilities['left'], 
                 df_abilities['right'], countries['spi_std'], 
                 names=df_abilities['names'],
                 title='Abilities Coefficient Plot',
                fig_size = (8,12))
plt.tight_layout()

### Let's Make Some Predictions! <a name="Pred"></a>

Here's what actually happened at the Semi-finals.

**The Semi-finals:** <br>
**1. England vs USA : 1-2**<br>
**2. Netherlands vs Sweden: 1-0**<br>

We are going to generate these matches using Stan and get the score-differentials. This is similar to what we did in the previous section: instead of "replaying" all the matches based on the tournament simulations, we are going to "play" the semi-finals and get the score differentials. Since we're a good month ahead of the events, we have the actual data for these matches and we get to assess how good our predictions are!

In [ ]:
import collections
semis_dict = collections.OrderedDict({'team_1': ['England', 'Netherlands'],
                 'team_2': ['USA', 'Sweden'],
                   'match_list': ['England vs USA', 'Netherlands vs Sweden'],
                  'actual_score_differential': [-1, 1]})

semis_dict['team_1'] = [ country_mapping.get(x) for x in semis_dict['team_1']]
semis_dict['team_2'] = [ country_mapping.get(x) for x in semis_dict['team_2']]
semis_dict

In [ ]:
#Replace team names by their rankings;
team1 = [ country_mapping.get(x) for x in matches['team_1']]
team2 = [ country_mapping.get(x) for x in matches['team_2']]

new_dict = collections.OrderedDict({'I': I,
               'N': N,
                'team_1': team1,
               'team_2': team2,
               'score_1': matches['score_1'].values,
               'score_2': matches['score_2'].values,
              'spi_std': countries['spi_std'].values,
               'team_1_semis':[6, 5],
                'team_2_semis':[1, 10]})



In [ ]:
new_dict

In [ ]:
stan_wc_semis = Model(stan_file='models/worldcup_semis.stan')
stan_wc_semis.compile()
stan_wc_semis

In [ ]:
worldcup_semis_fit = stan_wc_semis.sample(data=new_dict, chains=4)

In [ ]:
worldcup_semis_fit.diagnose()
worldcup_semis_fit.summary()

In [ ]:
#We can re-do our entire analysis discussed in the previous section, but just for these two matches!
df_semis = worldcup_semis_fit.summary().round(decimals = 2)
dft_semis = df_semis.transpose()
semi_filter = [col for col in dft_semis if col.startswith('semi')]
td_semis = dft_semis[semi_filter]

In [ ]:
df_teamdifferentials_semis = pd.DataFrame({'midway': td_semis.loc['50%'].values,
                                       'names':semis_dict['match_list']})
df_teamdifferentials_semis.loc[:, 'left'] = td_semis.loc['5%'].values
df_teamdifferentials_semis.loc[:, 'right'] = td_semis.loc['95%'].values

In [ ]:
coefficient_plot(df_teamdifferentials_semis['midway'], 
                          df_teamdifferentials_semis['left'], 
                          df_teamdifferentials_semis['right'], 
                          np.array(semis_dict['actual_score_differential']),
                          names=df_teamdifferentials_semis['names'], 
                          title='Game Differentials for the Semi-finals', 
                          fig_size = (9,10))
plt.tight_layout()